In [1]:
from glob import glob
import os

import cv2
import numpy as np
import torch
from PIL import Image
from pdf2image import convert_from_path
from imutils import resize
from tqdm import tqdm
from scoring import *
from border_and_title import *

good = list(glob("../data/original/Good/*.pdf"))
ugly = list(glob("../data/original/Ugly/*.pdf"))

len(good), len(ugly)

(16, 25)

In [2]:
def process(files, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2

    progress = tqdm(total=len(files))

    for f in files:
        pages = convert_from_path(f, dpi=300)
        img = np.array(pages[0])
        img = im_resize(img)
        img_comp = img.copy()

        im_h, im_w, _ = img.shape
        sorted_indices, contours, hierarchy_area = detect_objects(img)
        title_contours, line_90, data = detect_probable_title_sections(img, return_states=True)
        ocr_result = data["ocr_result"]
        mask = data["mask"]
        inner_border_lines = data["inner_border_lines"]  #  0: Bottom, 1: Top, 2: Right, 3: Left
        words = process_text(ocr_result, im_h, im_w)

        border_1, border_2 = detect_borders(contours, sorted_indices, words)
        contours = detect_text_tables(img, words, mask)
        title_contours = title_contours + contours

        boundary = get_boundary(border_1, border_2, inner_border_lines)
        title_boundary = get_title_boundary(boundary, line_90, title_contours, words, im_h)

        intersected_extended_lines, boundary_lines, boundary_title_block_lines, intersected_points = detect_intersection_with_borders(img.copy(), boundary, title_boundary, return_states=True)

        intersected_words = detect_intersected_texts(words, boundary_lines, boundary_title_block_lines)
        for idx, row in intersected_words.iterrows():
            bx1, by1, bx2, by2 = row[["x1", "y1", "x2", "y2"]].astype(int)
            cv2.rectangle(img_comp, (bx1, by1), (bx2, by2), (0, 0, 255), 2)

        cv2.rectangle(img_comp, boundary[0], boundary[1], color, thickness)
        cv2.rectangle(img_comp, title_boundary[0], title_boundary[1], color, thickness)

        # for x1, y1, x2, y2 in intersected_extended_lines:
        #     cv2.line(img_comp, (x1, y1), (x2, y2), (0, 0, 255), 2)

        for x, y in intersected_points:
            cv2.circle(img_comp, (int(x), int(y)), 20, (0, 0, 255), 2)

        cv2.imwrite(f"{output_dir}/{os.path.basename(f)}.jpg", np.hstack((img, img_comp)))
        progress.update(1)

In [3]:
process(ugly, "../data/intersection-with-boundary/ugly")
process(good, "../data/intersection-with-boundary/good")

 28%|██▊       | 7/25 [00:07<00:20,  1.13s/it]/home/sadid/anaconda3/lib/python3.12/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (139513096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 96%|█████████▌| 24/25 [00:25<00:01,  1.04s/it]/home/sadid/anaconda3/lib/python3.12/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (139513096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 16/16 [00:28<00:00,  1.79s/it]
